# Python Sample

In [12]:
import os
import grpc
import time
import sched
import csv
import logging

from IterableQueue import IterableQueue
from EventThread import EventThread

from remote.TMSRemote_pb2_grpc import *
from remote.TMSRemoteCommon_pb2 import *
from remote.TMSRemoteRequests_pb2 import *
from remote.TMSTradingRequests_pb2 import *

In [13]:
# Server and logon prameters 
SERVER = 'localhost'
GRPCPORT = '8083'
USER = 'demo'
PASSWORD = ''

#additional init

FILE_NAME = 'grpcSample.csv'
STRING_FIELDS = set(['Instrument', 'ClientName', 'SetPxTo'])

directory = os.getcwd()

logging_format = '%(asctime)s %(levelname)-6s %(message)s'
logging.basicConfig(level=logging.INFO, format=logging_format)
fh = logging.FileHandler(directory + '/sample.log')
fh.setFormatter(logging.Formatter(logging_format))
logging.getLogger('').addHandler(fh)




In [14]:
# Connect to server

ssl_credentials = grpc.ssl_channel_credentials(open(directory + '/cert.pem', 'rb').read())
channel = grpc.secure_channel(SERVER + ':' + GRPCPORT, ssl_credentials)
client = TMSRemoteStub(channel)

In [15]:
# LOGIN:

client.login(LoginRequest(user=USER, password=PASSWORD))


In [16]:
# create market portfolio

PORTFOLIO = 'simpleTest - ' + time.strftime('%Y%d%m %H:%M:%S')


try:
    client.createMarketPortfolio(CreateMarketPortfolioRequest(name=PORTFOLIO, type=PortfolioType.Pure))
except grpc.RpcError as e:
    error_code = dict(e.initial_metadata()).get('errorcode', None)
    if error_code != 'CannotCreatePortfolio':
        raise e
    logging.info('Portfolio already exists')

# load targets from CSV file
instruments = set()
targets = []
with open(directory + '/' + FILE_NAME, 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        target = {'stringFields': {k: v for k, v in row.items() if k in STRING_FIELDS},
                  'numericFields': {k: float(v) for k, v in row.items() if k not in STRING_FIELDS}}
        targets.append(target)
        instruments.add(row['Instrument'])

# create market targets
target_ids = client.addMarketTargets(AddMarketTargetsRequest(
    portfolio=PORTFOLIO,
    fields=targets
)).targetId


In [17]:
# subscribe for market targets

def market_target_event_processor(event):
    event_case = event.WhichOneof('event')
    if event_case == 'added':
        added = event.added
        target_id = added.targetId
        logging.info('New target ' + str(target_id )+ ' added')
    elif event_case == 'updated':
        updated = event.updated
        logging.info('Target ' + str(updated.targetId )+ ' updated')


market_target_subscription_queue = IterableQueue(SubscribeForTargetsRequest(
    filter="Portfolio = '" + PORTFOLIO + "'",
    field=['TgtID', 'Unreleased', 'Text']
))
events = client.subscribeForMarketTargets(iter(market_target_subscription_queue))
EventThread(events, market_target_event_processor).start()

2020-09-29 16:17:58,193 INFO   New target 18156 added
2020-09-29 16:17:58,196 INFO   New target 18157 added


In [18]:
""" add another target to an exisiting portfolio """
target_id = client.addMarketTargets(AddTargetsRequest(
    portfolio=PORTFOLIO,
    fields=[{
        'stringFields': {'Instrument': 'MSFT', 'TrnDestination': 'Simulator1'},
        'numericFields': {'Side': Side_Buy, 'TgtQty': 1500, 'TgtOrdType': OrdType_Market, 'WaveSizeType': WaveSizeType_PctTgtQty, 'WaveSize': 100}}
    ]
)).targetId[0]


2020-09-29 16:18:01,197 INFO   New target 18158 added


In [19]:
""" modify market target """
client.modifyMarketTargets(ModifyTargetsRequest(
    targetId=[target_id],
    fields=[{
        'stringFields': {'TrnDestination': 'Simulator1'},
        'numericFields': {'WaveSizeType': WaveSizeType_PctTgtQty, 'WaveSize': 50}}
    ]
))

In [20]:
""" send target order with additional field(s) in the message """
client.sendOrders(SendOrdersRequest(
    targetId=[target_id],
    message=[{
        'stringFields': {FIXTag_Text: 'NOFILL'}}
    ]
))

2020-09-29 16:18:04,201 INFO   Target 18158 updated
